In [1]:
tmp <-read.table('handovercompiled.Yanis.080919.csv',sep=",",header=T)
rownames(tmp) <- tmp$data_pd
master <- read.table('../../../data/initial_dataset/Master_04_10_2019.csv',sep=",",header=T)
rownames(master) <- master$data_pd

In [17]:
colnames(tmp)

[1] "donor_id"                "data_pd"                
 [3] "sample_pd.x"             "date_entry"             
 [5] "trtcode"                 "AMLID"                  
 [7] "dead60"                  "MRDgp1"                 
 [9] "MRDgp2"                  "CRby1"                  
[11] "CRby2"                   "MRD1"                   
[13] "MRD2"                    "LAIPdetection1"         
[15] "LAIPvalue1"              "Sample1"                
[17] "DiagnosticLAIPscreening" "LAIPdetection2"         
[19] "LAIPvalue2"              "Sample2"                
[21] "LAIPassessed1"           "LAIPassessed2"          
[23] "trial.y"                 "sample_pd.y"            
[25] "OS_randomization"        "DEADStatus"             
[27] "RFSyears"                "RFSStatus"              
[29] "RelapseCIStatus"         "DaystoFirstCR"          
[31] "CODCause"                "OS_CR"                  
[33] "CRdate"                  "randDate_PB"            
[35] "CRDate_PB"               "relapseDate_PB"         
[37] "boneDate_PB"             "longCR_PB"

In [3]:
head(master[,colnames(tmp)])

,donor_id,data_pd,sample_pd.x,date_entry,trtcode,AMLID,dead60,MRDgp1,MRDgp2,CRby1,⋯,RelapseCIStatus,DaystoFirstCR,CODCause,OS_CR,CRdate,randDate_PB,CRDate_PB,relapseDate_PB,boneDate_PB,longCR_PB
PD20338c,11-3496,PD20338c,PD20338c,1994-01-02,110061,NA,NA,NA,NA,NA,⋯,1,27,INFECTION,1.9329230,1994-02-28,0,0,0,0,0
PD20337c,11-3807,PD20337c,PD20337c,1995-09-08,110054,NA,NA,NA,NA,NA,⋯,2,NA,RESISTANT DIS,NA,NA,0,0,0,0,0
PD20336a,11-3885,PD20336a,PD20336a,1995-12-18,110036,NA,NA,NA,NA,NA,⋯,2,NA,INFECT + HAEM,NA,NA,0,0,0,0,0
PD20335c,11-4052,PD20335c,PD20335c,1996-11-14,110045,NA,NA,NA,NA,NA,⋯,2,NA,INFECTION,NA,NA,0,0,0,0,0
PD20334a,11-4169,PD20334a,PD20334a,1997-06-18,110058,NA,NA,NA,NA,NA,⋯,1,27,RECURRENT LEUK,0.6078029,1997-07-15,0,0,0,0,0
PD20333c,12-5272,PD20333c,PD20333c,1995-03-11,NA,NA,NA,NA,NA,NA,⋯,1,88,RECURRENT LEUK,1.5523610,1996-01-30,0,0,0,0,0


In [23]:
sum(table(master[,colnames(tmp)]$AMLID))

[1] 1167

In [24]:
sum(table(master[,colnames(tmp)]$AMLID))

[1] 1167

In [12]:
dim(master)

[1] 2428  522